In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets, interact
from IPython.display import display, Image, HTML
from datetime import datetime

# Konfigurasi API Key
api_key = 'f6a79db0fba22353d1625b676200f157'

# Fungsi ambil data cuaca realtime
def get_weather(city_name):
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {'q': city_name, 'appid': api_key, 'units': 'metric'}
    response = requests.get(url, params=params)
    data = response.json()
    
    if response.status_code == 200:
        icon_code = data['weather'][0]['icon']
        weather_info = {
            'Kota': data['name'],
            'Suhu (°C)': data['main']['temp'],
            'Cuaca': data['weather'][0]['description'].capitalize(),
            'Kelembaban (%)': data['main']['humidity'],
            'Angin (m/s)': data['wind']['speed'],
            'Tekanan (hPa)': data['main']['pressure'],
            'Icon': f"http://openweathermap.org/img/wn/{icon_code}@2x.png"
        }
        return weather_info
    else:
        return None

# Fungsi ambil data forecast
def get_forecast(city_name):
    url = "http://api.openweathermap.org/data/2.5/forecast"
    params = {'q': city_name, 'appid': api_key, 'units': 'metric'}
    response = requests.get(url, params=params)
    data = response.json()

    if response.status_code == 200:
        forecast_list = []
        for item in data['list']:
            forecast_list.append({
                'Waktu': datetime.fromtimestamp(item['dt']),
                'Suhu (°C)': item['main']['temp'],
                'Kelembaban (%)': item['main']['humidity'],
                'Angin (m/s)': item['wind']['speed']
            })
        df = pd.DataFrame(forecast_list)
        return df
    else:
        return None

# Tampilkan info cuaca dalam card-style
def display_weather_card(data):
    html = f"""
    <div style="background: #3498db; padding: 20px; border-radius: 15px; color: white; width: 350px; margin:10px 0;">
        <h2 style="margin:0;">{data['Kota']}</h2>
        <img src="{data['Icon']}" style="vertical-align: middle;">
        <h3>{data['Cuaca']}</h3>
        <p style="font-size:16px;">
            🌡️ Suhu: {data['Suhu (°C)']}°C<br>
            💧 Kelembaban: {data['Kelembaban (%)']}%<br>
            💨 Angin: {data['Angin (m/s)']} m/s<br>
            🔵 Tekanan: {data['Tekanan (hPa)']} hPa
        </p>
    </div>
    """
    display(HTML(html))

# Plot forecast 3 parameter sekaligus
def plot_forecast_chart(forecast_df, city):
    plt.figure(figsize=(12,6))
    sns.lineplot(x='Waktu', y='Suhu (°C)', data=forecast_df, marker='o', label='Suhu (°C)')
    sns.lineplot(x='Waktu', y='Kelembaban (%)', data=forecast_df, marker='s', label='Kelembaban (%)')
    sns.lineplot(x='Waktu', y='Angin (m/s)', data=forecast_df, marker='^', label='Angin (m/s)')
    plt.title(f'Perkiraan Cuaca 5 Hari di {city}')
    plt.xlabel('Waktu')
    plt.ylabel('Nilai')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Simpan data ke file
def save_to_file(df, filename_base):
    df.to_csv(f'{filename_base}.csv', index=False)
    df.to_excel(f'{filename_base}.xlsx', index=False)
    print(f"✔️ Data disimpan ke {filename_base}.csv & {filename_base}.xlsx")

# Aplikasi utama interaktif
def interactive_weather_dashboard():
    cities_input = widgets.Text(
        value='Jakarta, Bandung, Surabaya',
        description='Kota:',
        placeholder='Pisahkan dengan koma'
    )
    display(cities_input)

    button = widgets.Button(description="Tampilkan Data Cuaca")
    display(button)

    def on_button_clicked(b):
        cities = [city.strip() for city in cities_input.value.split(',')]
        weather_data = []
        for city in cities:
            data = get_weather(city)
            if data:
                weather_data.append(data)
            else:
                print(f"⚠️ Data untuk {city} tidak ditemukan.")

        df = pd.DataFrame(weather_data)
        display(df.style.background_gradient(cmap='Blues'))

        # Simpan ke file
        save_to_file(df, 'data_cuaca_summary')

        # Dropdown pilih kota
        dropdown = widgets.Dropdown(options=cities, description='Detail:')
        display(dropdown)

        def on_dropdown_change(change):
            if change['type'] == 'change' and change['name'] == 'value':
                selected_city = change['new']
                data = get_weather(selected_city)
                forecast_df = get_forecast(selected_city)
                if data:
                    display_weather_card(data)
                    if forecast_df is not None:
                        plot_forecast_chart(forecast_df, selected_city)
                    else:
                        print("Data forecast tidak tersedia.")

        dropdown.observe(on_dropdown_change)

    button.on_click(on_button_clicked)

# Jalankan dashboard cuaca interaktif
interactive_weather_dashboard()


Text(value='Jakarta, Bandung, Surabaya', description='Kota:', placeholder='Pisahkan dengan koma')

Button(description='Tampilkan Data Cuaca', style=ButtonStyle())